# Résumé

Ce projet vise à automatiser le suivi des prix d'un produit spécifique sur Amazon. Il utilise des outils de web scraping comme requests et BeautifulSoup pour extraire les informations du produit, notamment son titre et son prix. En cas de détection d'un CAPTCHA, le script utilise Selenium pour contourner l'obstacle. Chaque jour, le prix est vérifié, puis ajouté à un fichier CSV pour un suivi historique.

Lorsque le prix du produit tombe sous un seuil déterminé (750€ dans ce cas), une alerte par e-mail est envoyée à l'utilisateur pour lui signaler l'opportunité d'achat. Le processus est entièrement automatisé, avec une vérification quotidienne grâce à une boucle qui permet de vérifier les prix toutes les 24 heures, sans nécessiter d'intervention manuelle.

## Importation des bibliothèques nécessaires

In [28]:
from bs4 import BeautifulSoup
import requests
import smtplib
import time
import datetime
import csv
import pandas as pd
from selenium import webdriver

Ces bibliothèques sont utilisées pour différentes parties du processus, comme la récupération des données d'une page web, l'analyse de contenu HTML, l'envoi d'e-mails et la gestion des fichiers CSV.  

- **BeautifulSoup** : Pour l'analyse de contenu HTML.  
- **requests** : Pour récupérer le contenu des pages web.  
- **smtplib** : Pour envoyer des e-mails.  
- **time** : Pour créer des pauses dans le code.  
- **datetime** : Pour obtenir la date actuelle.  
- **csv** : Pour gérer les fichiers CSV.  
- **pandas** : Pour faciliter la lecture et la gestion des données CSV.  


## Récupération et analyse de la page web

In [12]:
# URL du produit
url = 'https://www.amazon.fr/Apple-iPhone-15-128-Go/dp/B0CHXFCYCR'

# En-têtes HTTP pour simuler une requête de navigateur
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36",
    "Accept-Encoding": "gzip, deflate",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
    "DNT": "1",
    "Connection": "close",
    "Upgrade-Insecure-Requests": "1"
}

# Récupérer le contenu de la page via requests
response = requests.get(url, headers=headers)

- **L'URL de la page produit Amazon** est définie pour accéder à une ressource spécifique, ici la page d'un iPhone 15.  

- Les **en-têtes HTTP** sont configurés pour simuler une requête provenant d'un navigateur (via le champ **User-Agent** et autres paramètres) afin d'éviter que le site identifie la requête comme un bot et la bloque.  

- Une **requête HTTP GET** est envoyée à l'URL à l'aide de la bibliothèque **requests**, ce qui permet de récupérer le contenu brut (**HTML**) de la page.


**Note :**
Dans ce projet, j'ai utilisé le service *httpbin.org* pour consulter les en-têtes de requêtes HTTP. En accédant à l'URL <ins>httpbin.org/get</ins>, le site renvoie une réponse JSON contenant les en-têtes (headers) envoyés avec la requête.

In [13]:
# Si un CAPTCHA est détecté, utiliser Selenium
if "captcha" in response.text.lower():
    print("CAPTCHA détecté. Utilisation de Selenium pour contourner.")
    
    # Initialisation du driver Selenium
    driver = webdriver.Chrome()  # Assurez-vous d'avoir ChromeDriver installé et dans votre PATH
    driver.get(url)
    
    # Demander à l'utilisateur de résoudre le CAPTCHA
    input("Veuillez résoudre le CAPTCHA dans le navigateur, puis appuyez sur Entrée.")
    
    # Récupérer le HTML après la résolution du CAPTCHA
    page_source = driver.page_source
    driver.quit()
    html_content = page_source
else:
    # Si aucun CAPTCHA, utiliser la réponse de requests
    html_content = response.content

CAPTCHA détecté. Utilisation de Selenium pour contourner.


Veuillez résoudre le CAPTCHA dans le navigateur, puis appuyez sur Entrée. 


## Extraction des données du produit (titre et prix)

In [15]:
# Analyser le contenu HTML avec BeautifulSoup
soup = BeautifulSoup(html_content, "html.parser")
soup

<html class="a-js a-audio a-video a-canvas a-svg a-drag-drop a-geolocation a-history a-webworker a-autofocus a-input-placeholder a-textarea-placeholder a-local-storage a-gradients a-transform3d -scrolling a-text-shadow a-text-stroke a-box-shadow a-border-radius a-border-image a-opacity a-transform a-transition null" data-19ax5a9jf="dingo" data-aui-build-date="3.24.10-2024-12-17" lang="fr-fr"><!-- sp:feature:head-start --><head><script async="" crossorigin="anonymous" src="https://images-eu.ssl-images-amazon.com/images/I/31bJewCvY-L.js"></script><style class="vjs-styles-defaults">
      .video-js {
        width: 300px;
        height: 150px;
      }

      .vjs-fluid {
        padding-top: 56.25%
      }
    </style><style class="vjs-styles-dimensions">
      .detailpage-imageblock-player-7f6d6913-bd5d-4940-b6be-51116595646a-container-element-dimensions {
        width: 640px;
        height: 360px;
      }

      .detailpage-imageblock-player-7f6d6913-bd5d-4940-b6be-51116595646a-conta

In [16]:
# Extraire le titre du produit
title = soup.find(id='productTitle').get_text().strip() if soup.find(id='productTitle') else "Titre introuvable"

In [17]:
title

'Apple iPhone 15 (128 Go) - Noir'

In [22]:
# Extraire le prix du produit et enlever les espaces
price = soup.find(id='tp_price_block_total_price_ww')
if price:
    price = price.get_text().strip()[:6]  # Prendre les 6 premiers caractères du prix
    price = price.replace(' ', '')  # Supprimer les espaces éventuels
    price = price.replace(',', '.')  # Remplacer la virgule par un point
else:
    price = "Prix introuvable"

# Conversion en float si le prix est valide
try:
    price_float = float(price)
    print(f"Prix du produit en float : {price_float}")
except ValueError:
    print(f"Le prix est invalide ou introuvable : {price}")

Prix du produit en float : 772.0


## Récupération de la date actuell et ajouter les données dans un fichier CSV

In [23]:
# Obtenir la date actuelle
today = datetime.date.today()

In [26]:
header = ['Title','Price','Date'] # Définir l'en-tête du fichier CSV
data = [title, price, today]# Ajouter les données extraites dans un fichier CSV

with open('AmazonWebScraperDataset.csv','w',newline='', encoding='UTF8') as f:
    writer = csv.writer(f)
    writer.writerow(header)
    writer.writerow(data)

In [29]:
# Lire le fichier CSV pour vérifier son contenu
df = pd.read_csv('AmazonWebScraperDataset.csv')
df

,Title,Price,Date
0,Apple iPhone 15 (128 Go) - Noir,772.0,2024-12-24


In [34]:
# Ajouter de nouvelles données au fichier CSV (en cas de nouvelles vérifications)
with open('AmazonWebScraperDataset.csv', 'a+', newline='', encoding='UTF8') as f:
    writer = csv.writer(f)
    writer.writerow(data)

In [35]:
df = pd.read_csv('AmazonWebScraperDataset.csv')
df

,Title,Price,Date
0,Apple iPhone 15 (128 Go) - Noir,772.0,2024-12-24
1,Apple iPhone 15 (128 Go) - Noir,772.0,2024-12-24
2,Apple iPhone 15 (128 Go) - Noir,772.0,2024-12-24
3,Apple iPhone 15 (128 Go) - Noir,772.0,2024-12-24


## Fonction d'envoi d'email

In [37]:
def envoyer_mail():
        # Connexion au serveur SMTP de Gmail
        server = smtplib.SMIP_SSL('smtp.gmail.com',465)
        server.ehlo()
    
        # Connexion avec les identifiants
        server.login('nouhaila@gmail.com','xxxpasswordxxx')
        # Sujet et corps du mail
        subject = "Le téléphone que vous voulez est en dessous de 810€ ! C'est votre chance d'acheter !"
        body = "Nouhaila, c'est le moment que nous attendions, n'attendez plus pour acheter le téléphone de vos rêves !"

        # Créer le message à envoyer
        msg = f"Subject: {subject}\n\n{body}"
    
        # Envoyer le mail
        server.sendmail(
            'nouhaila@gmail.com',
            msg
            )

In [38]:
# Définir la fonction qui vérifie les prix et envoie un mail si le prix est sous le seuil
def verifier_prix():
    # Requête pour récupérer la page produit
    page = requests.get(URL, headers=headers)
    soup1 = BeautifulSoup(page.content, "html.parser")
    soup2 = BeautifulSoup(soup1.prettify(), "html.parser")
    
    # Extraire le titre et le prix du produit
    title = soup2.find(id='productTitle').get_text()
    price = soup2.find(id='tp_price_block_total_price_ww').get_text()
    price = price.strip()[:6]
    title = title.strip()
    
    # Obtenir la date actuelle
    today = datetime.date.today()
    
    # Ajouter les nouvelles données dans le fichier CSV
    header = ['Titre', 'Prix', 'Date']
    data = [title, price, today]

    with open('AmazonWebScraperDataset.csv', 'a+', newline='', encoding='UTF8') as f:
        writer = csv.writer(f)
        writer.writerow(data)

    # Si le prix est inférieur à 750€, envoyer un email
    if float(price.replace("€", "").replace(",", ".")) < 750:
        envoyer_mail()

## Vérification automatique du prix tous les jours

In [ ]:
while True:
    verifier_prix()
    time.sleep(86400)  # Délai de 86400 secondes (24 heures)

In [ ]:
# Lire le fichier CSV après chaque itération pour vérifier les nouvelles données
df = pd.read_csv('AmazonWebScraperDataset.csv')
df